# Academic Benchmarking Data Generator

This notebook generates high-quality sample ground truth data for academic IR and NLU benchmarking based on the actual knowledge graph structure and domain knowledge.

In [ ]:
import os
import sys
import json
import pandas as pd
import numpy as np
import random
from typing import List, Dict, Tuple
import re

# Add parent directory to path
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath('.')))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from knowledge_graph_module.kg_builder import RANNeo4jIntegrator
from chatbot_module.chatbot import EnhancedRANChatbot

## Setup and Configuration

In [ ]:
# Neo4j connection settings
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "neo4j"

# Initialize connections
try:
    integrator = RANNeo4jIntegrator(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    chatbot = EnhancedRANChatbot(integrator, use_domain_model=True)
    print("✅ Connected to Neo4j and initialized chatbot")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("Please ensure Neo4j is running and accessible")

## Explore Knowledge Graph Structure

In [ ]:
# Get schema overview
schema = integrator.get_schema_overview()
print(f"Total tables: {schema.get('tables', {}).get('total_tables', 0)}")
print(f"Total columns: {schema.get('columns', {}).get('total_columns', 0)}")
print(f"Total concepts: {len(schema.get('concepts', []))}")

# Get sample tables for analysis
sample_tables = schema.get('tables', {}).get('sample_tables', [])[:20]
print(f"\nSample tables ({len(sample_tables)}):")
for table in sample_tables:
    print(f"- {table['table_name']} ({table['row_count']} rows)")

In [ ]:
# Get domain concepts for better query generation
concepts = schema.get('concepts', [])
print(f"Available concepts ({len(concepts)}):")
for concept in concepts[:15]:
    print(f"- {concept}")

# Get relationship patterns
relationships = schema.get('relationships', [])
print(f"\nRelationship patterns ({len(relationships)}):")
for rel in relationships[:10]:
    print(f"- {rel['type']}: {rel['count']} instances")

## Define Query Templates and Patterns

In [ ]:
# RAN domain-specific query templates
QUERY_TEMPLATES = {
    'power_analysis': [
        "Show me power consumption data from {table}",
        "Find energy efficiency metrics in {table}.{column}",
        "Get power optimization insights for {concept}",
        "Display power measurements from {table}",
        "Analyze energy consumption patterns in {concept}"
    ],
    'frequency_management': [
        "Show frequency allocation data from {table}",
        "Find spectrum usage in {table}.{column}",
        "Get carrier frequency information from {concept}",
        "Display frequency band data in {table}",
        "Analyze spectrum efficiency for {concept}"
    ],
    'performance_metrics': [
        "Show throughput data from {table}",
        "Find KPI measurements in {table}.{column}",
        "Get performance analysis for {concept}",
        "Display quality metrics from {table}",
        "Analyze network performance in {concept}"
    ],
    'cell_configuration': [
        "Show cell parameters from {table}",
        "Find configuration settings in {table}.{column}",
        "Get cell setup information for {concept}",
        "Display antenna configuration from {table}",
        "Analyze cell optimization in {concept}"
    ],
    'neighbor_relations': [
        "Show neighbor cell data from {table}",
        "Find handover information in {table}.{column}",
        "Get neighbor relations for {concept}",
        "Display adjacency data from {table}",
        "Analyze cell relationships in {concept}"
    ],
    'timing_sync': [
        "Show timing synchronization from {table}",
        "Find sync parameters in {table}.{column}",
        "Get timing data for {concept}",
        "Display synchronization status from {table}",
        "Analyze timing accuracy in {concept}"
    ]
}

# Common RAN table patterns
RAN_TABLE_PATTERNS = {
    'power': ['power', 'energy', 'consumption', 'dbm', 'watt'],
    'frequency': ['frequency', 'freq', 'spectrum', 'band', 'carrier', 'eutra'],
    'performance': ['throughput', 'kpi', 'quality', 'performance', 'metric'],
    'cell': ['cell', 'sector', 'site', 'antenna', 'config'],
    'neighbor': ['neighbor', 'relation', 'handover', 'adjacency'],
    'sync': ['sync', 'timing', 'synchronization', 'time'],
    'measurement': ['measurement', 'monitor', 'report', 'sample'],
    'optimization': ['optimization', 'tuning', 'adjustment', 'parameter']
}

print("Query templates and patterns defined")
print(f"Templates: {list(QUERY_TEMPLATES.keys())}")
print(f"Patterns: {list(RAN_TABLE_PATTERNS.keys())}")

## Generate IR Ground Truth Data

In [ ]:
def categorize_table(table_name: str, table_info: dict = None) -> str:
    """Categorize table based on name and content"""
    name_lower = table_name.lower()
    
    for category, keywords in RAN_TABLE_PATTERNS.items():
        if any(keyword in name_lower for keyword in keywords):
            return category
    
    return 'general'

def find_related_tables(target_table: str, all_tables: list, max_related: int = 5) -> list:
    """Find tables related to the target table"""
    category = categorize_table(target_table)
    related = []
    
    for table in all_tables:
        if table['table_name'] != target_table:
            if categorize_table(table['table_name']) == category:
                related.append(table['table_name'])
    
    # Add some cross-category relationships for realism
    cross_category_maps = {
        'power': ['performance', 'cell'],
        'frequency': ['performance', 'cell'],
        'cell': ['neighbor', 'performance'],
        'performance': ['power', 'frequency']
    }
    
    if category in cross_category_maps:
        for cross_cat in cross_category_maps[category]:
            for table in all_tables:
                if categorize_table(table['table_name']) == cross_cat:
                    related.append(table['table_name'])
    
    return list(set(related))[:max_related]

def generate_ir_ground_truth(num_queries: int = 50) -> pd.DataFrame:
    """Generate IR ground truth data"""
    ir_data = []
    tables = schema.get('tables', {}).get('sample_tables', [])
    concepts = schema.get('concepts', [])
    
    random.seed(42)  # For reproducibility
    
    for i in range(num_queries):
        # Select random category and template
        category = random.choice(list(QUERY_TEMPLATES.keys()))
        template = random.choice(QUERY_TEMPLATES[category])
        
        # Select target table based on category
        category_pattern = category.split('_')[0] if '_' in category else category
        relevant_tables = []
        
        # Find tables matching the category
        for table in tables:
            table_category = categorize_table(table['table_name'])
            if (table_category == category_pattern or 
                category_pattern in table['table_name'].lower() or
                any(keyword in table['table_name'].lower() 
                    for keyword in RAN_TABLE_PATTERNS.get(category_pattern, []))):
                relevant_tables.append(table['table_name'])
        
        if not relevant_tables:
            # Fallback to random table
            target_table = random.choice(tables)['table_name']
            relevant_tables = [target_table]
        else:
            target_table = random.choice(relevant_tables)
        
        # Generate query
        if '{table}' in template:
            query = template.format(table=target_table)
        elif '{concept}' in template:
            concept = random.choice(concepts) if concepts else target_table
            query = template.format(concept=concept)
            # Find tables related to this concept
            concept_tables = []
            for table in tables:
                if (concept.lower() in table['table_name'].lower() or
                    any(word in table['table_name'].lower() for word in concept.lower().split())):
                    concept_tables.append(table['table_name'])
            relevant_tables = concept_tables if concept_tables else relevant_tables
        elif '{column}' in template:
            # Get a sample column from target table
            try:
                table_details = integrator.get_table_details(target_table)
                columns = table_details.get('columns', [])
                if columns:
                    sample_column = random.choice(columns)['name']
                    query = template.format(table=target_table, column=sample_column)
                else:
                    query = template.replace('.{column}', '').format(table=target_table)
            except:
                query = template.replace('.{column}', '').format(table=target_table)
        else:
            query = template
        
        # Find additional related tables
        related_tables = find_related_tables(target_table, tables)
        all_relevant = list(set([target_table] + relevant_tables[:2] + related_tables[:3]))
        
        ir_data.append({
            'query': query,
            'relevant_tables': ','.join(all_relevant[:5]),  # Limit to top 5
            'primary_table': target_table,
            'category': category,
            'num_relevant': len(all_relevant)
        })
    
    return pd.DataFrame(ir_data)

# Generate IR data
ir_df = generate_ir_ground_truth(50)
print(f"Generated {len(ir_df)} IR queries")
print(f"Categories: {ir_df['category'].value_counts().to_dict()}")
ir_df.head(10)

## Generate NLU Ground Truth Data

In [ ]:
def generate_expected_answer(query: str, primary_table: str, category: str) -> str:
    """Generate expected answer based on query and context"""
    
    # Template responses based on category
    answer_templates = {
        'power_analysis': [
            f"📊 Power analysis from {primary_table} shows energy consumption patterns across network elements. Key metrics include power efficiency ratios and consumption trends.",
            f"⚡ Energy data from {primary_table} reveals optimization opportunities in power management systems with detailed consumption measurements.",
            f"🔋 Power consumption analysis indicates efficiency levels from {primary_table} with recommendations for energy optimization."
        ],
        'frequency_management': [
            f"📡 Frequency data from {primary_table} displays spectrum allocation and carrier management information across bands.",
            f"🌐 Spectrum analysis from {primary_table} shows frequency utilization patterns and band efficiency metrics.",
            f"📊 Carrier frequency information from {primary_table} includes allocation status and spectrum optimization insights."
        ],
        'performance_metrics': [
            f"📈 Performance metrics from {primary_table} show throughput, latency, and quality indicators across network elements.",
            f"🎯 KPI analysis from {primary_table} reveals network performance trends and optimization opportunities.",
            f"📊 Quality measurements from {primary_table} include RSRP, RSRQ, and throughput performance data."
        ],
        'cell_configuration': [
            f"🏗️ Cell configuration from {primary_table} shows parameter settings, antenna configurations, and optimization values.",
            f"⚙️ Configuration data from {primary_table} includes cell parameters, thresholds, and antenna settings.",
            f"📋 Cell setup information from {primary_table} displays optimization parameters and configuration status."
        ],
        'neighbor_relations': [
            f"🔗 Neighbor relations from {primary_table} show cell adjacency patterns and handover optimization data.",
            f"🤝 Handover analysis from {primary_table} reveals neighbor cell relationships and mobility patterns.",
            f"📊 Cell relationship data from {primary_table} includes neighbor configurations and handover statistics."
        ],
        'timing_sync': [
            f"⏰ Timing synchronization from {primary_table} shows sync accuracy and timing parameter configurations.",
            f"🕐 Sync data from {primary_table} reveals timing accuracy measurements and synchronization status.",
            f"📊 Timing analysis from {primary_table} includes sync parameters and accuracy metrics."
        ]
    }
    
    if category in answer_templates:
        return random.choice(answer_templates[category])
    else:
        return f"📋 Data from {primary_table} provides insights into {category.replace('_', ' ')} with relevant metrics and analysis."

def extract_entities_from_query(query: str, primary_table: str) -> list:
    """Extract key entities that should appear in responses"""
    entities = [primary_table]
    
    # Common RAN entities
    ran_entities = {
        'power': ['consumedEnergyMeasurement', 'powerOptimization', 'energyEfficiency'],
        'frequency': ['EUtranFrequency', 'freqBand', 'carrierFreq', 'spectrumAllocation'],
        'performance': ['throughputMeasurement', 'qualityIndicator', 'performanceCounter'],
        'cell': ['cellConfiguration', 'antennaConfig', 'cellParameters'],
        'neighbor': ['neighborRelation', 'handoverConfig', 'adjacencyList'],
        'sync': ['synchronizationConfig', 'timingAccuracy', 'syncStatus']
    }
    
    query_lower = query.lower()
    for category, entity_list in ran_entities.items():
        if category in query_lower:
            entities.extend(random.sample(entity_list, min(2, len(entity_list))))
    
    # Extract table.column patterns from query
    table_column_pattern = r'([A-Za-z][A-Za-z0-9_]*)\.[A-Za-z][A-Za-z0-9_]*'
    matches = re.findall(table_column_pattern, query)
    entities.extend(matches)
    
    return list(set(entities))[:5]  # Limit to 5 entities

def generate_nlu_ground_truth(ir_df: pd.DataFrame) -> pd.DataFrame:
    """Generate NLU ground truth based on IR queries"""
    nlu_data = []
    
    for _, row in ir_df.iterrows():
        query = row['query']
        primary_table = row['primary_table']
        category = row['category']
        
        # Generate expected answer
        answer = generate_expected_answer(query, primary_table, category)
        
        # Extract expected entities
        entities = extract_entities_from_query(query, primary_table)
        
        nlu_data.append({
            'query': query,
            'answer': answer,
            'entities': ','.join(entities),
            'intent': category,
            'complexity': 'medium' if len(entities) > 3 else 'simple',
            'domain': 'RAN'
        })
    
    return pd.DataFrame(nlu_data)

# Generate NLU data
nlu_df = generate_nlu_ground_truth(ir_df)
print(f"Generated {len(nlu_df)} NLU examples")
print(f"Intent distribution: {nlu_df['intent'].value_counts().to_dict()}")
print(f"Complexity distribution: {nlu_df['complexity'].value_counts().to_dict()}")
nlu_df.head(10)

## Validate Generated Data

In [ ]:
# Validate IR data
print("=== IR Data Validation ===")
print(f"Total queries: {len(ir_df)}")
print(f"Unique queries: {ir_df['query'].nunique()}")
print(f"Average relevant tables per query: {ir_df['num_relevant'].mean():.1f}")
print(f"Categories covered: {sorted(ir_df['category'].unique())}")

# Check for missing data
print(f"\nMissing data:")
print(f"- Queries with no relevant tables: {(ir_df['relevant_tables'] == '').sum()}")
print(f"- Queries with no primary table: {ir_df['primary_table'].isnull().sum()}")

# Sample validation
print(f"\nSample IR entries:")
for i, row in ir_df.sample(3).iterrows():
    print(f"Q: {row['query']}")
    print(f"   Relevant: {row['relevant_tables']}")
    print(f"   Primary: {row['primary_table']}")
    print()

In [ ]:
# Validate NLU data
print("=== NLU Data Validation ===")
print(f"Total examples: {len(nlu_df)}")
print(f"Unique queries: {nlu_df['query'].nunique()}")
print(f"Average entities per query: {nlu_df['entities'].apply(lambda x: len(x.split(',')) if x else 0).mean():.1f}")
print(f"Average answer length: {nlu_df['answer'].apply(len).mean():.0f} chars")

# Check for missing data
print(f"\nMissing data:")
print(f"- Queries with no answer: {nlu_df['answer'].isnull().sum()}")
print(f"- Queries with no entities: {(nlu_df['entities'] == '').sum()}")

# Sample validation
print(f"\nSample NLU entries:")
for i, row in nlu_df.sample(2).iterrows():
    print(f"Q: {row['query']}")
    print(f"A: {row['answer'][:100]}...")
    print(f"Entities: {row['entities']}")
    print(f"Intent: {row['intent']}")
    print()

## Test with Real Chatbot Responses

In [ ]:
# Test a few queries with the actual chatbot to ensure realism
print("=== Testing with Real Chatbot ===")

test_queries = ir_df.sample(3)['query'].tolist()

for query in test_queries:
    print(f"\nQuery: {query}")
    try:
        result = chatbot.enhanced_process_query(query)
        response = result.get('response') or chatbot.generate_response(result)
        print(f"Response: {response[:200]}...")
        print(f"Type: {result.get('type')}")
        
        # Extract retrieved tables
        retrieved_tables = []
        if result.get('type') == 'semantic_search':
            for r in (result.get('results') or [])[:5]:
                if r.get('table_name'):
                    retrieved_tables.append(r['table_name'])
        print(f"Retrieved tables: {retrieved_tables}")
        
    except Exception as e:
        print(f"Error: {e}")
    print("-" * 50)

## Save Generated Data

In [ ]:
# Save to CSV files
output_dir = os.path.dirname(os.path.dirname(os.path.abspath('.')))

# Save IR ground truth
ir_output_path = os.path.join(output_dir, 'sample_ir_ground_truth.csv')
ir_export_df = ir_df[['query', 'relevant_tables']].copy()
ir_export_df.to_csv(ir_output_path, index=False)
print(f"✅ IR ground truth saved to: {ir_output_path}")
print(f"   Shape: {ir_export_df.shape}")

# Save NLU ground truth
nlu_output_path = os.path.join(output_dir, 'sample_nlu_ground_truth.csv')
nlu_export_df = nlu_df[['query', 'answer', 'entities']].copy()
nlu_export_df.to_csv(nlu_output_path, index=False)
print(f"✅ NLU ground truth saved to: {nlu_output_path}")
print(f"   Shape: {nlu_export_df.shape}")

# Save detailed analysis data (optional)
ir_detailed_path = os.path.join(output_dir, 'detailed_ir_analysis.csv')
ir_df.to_csv(ir_detailed_path, index=False)
print(f"✅ Detailed IR analysis saved to: {ir_detailed_path}")

nlu_detailed_path = os.path.join(output_dir, 'detailed_nlu_analysis.csv')
nlu_df.to_csv(nlu_detailed_path, index=False)
print(f"✅ Detailed NLU analysis saved to: {nlu_detailed_path}")

## Generate Additional High-Quality Samples

In [ ]:
# Generate additional samples with more sophisticated patterns
def generate_advanced_ir_samples(num_samples: int = 25) -> pd.DataFrame:
    """Generate more sophisticated IR samples using actual table relationships"""
    advanced_samples = []
    
    # Get actual relationships from the KG
    try:
        relationships = integrator.get_table_relationships(limit=20)
    except:
        relationships = []
    
    # Advanced query patterns
    advanced_patterns = [
        "Correlate {table1} with {table2} for optimization insights",
        "Cross-reference {table1} and {table2} performance data",
        "Compare metrics between {table1} and {table2}",
        "Find relationships linking {table1} to {table2}",
        "Analyze joint patterns in {table1} and {table2}",
        "Show dependencies between {table1} and {table2}"
    ]
    
    tables = [t['table_name'] for t in schema.get('tables', {}).get('sample_tables', [])]
    
    for i in range(num_samples):
        if relationships and random.random() > 0.5:
            # Use actual relationship
            rel = random.choice(relationships)
            table1 = rel.get('table1', random.choice(tables))
            table2 = rel.get('table2', random.choice(tables))
        else:
            # Random pair
            table1, table2 = random.sample(tables, 2)
        
        pattern = random.choice(advanced_patterns)
        query = pattern.format(table1=table1, table2=table2)
        
        # Find related tables based on categories
        cat1 = categorize_table(table1)
        cat2 = categorize_table(table2)
        
        relevant_tables = [table1, table2]
        
        # Add tables from same categories
        for table in tables:
            if (categorize_table(table) in [cat1, cat2] and 
                table not in relevant_tables and 
                len(relevant_tables) < 6):
                relevant_tables.append(table)
        
        advanced_samples.append({
            'query': query,
            'relevant_tables': ','.join(relevant_tables),
            'primary_table': table1,
            'category': 'multi_table_analysis',
            'num_relevant': len(relevant_tables)
        })
    
    return pd.DataFrame(advanced_samples)

# Generate advanced samples
advanced_ir_df = generate_advanced_ir_samples(25)
print(f"Generated {len(advanced_ir_df)} advanced IR samples")

# Combine with original samples
combined_ir_df = pd.concat([ir_df, advanced_ir_df], ignore_index=True)
print(f"Total IR samples: {len(combined_ir_df)}")

# Generate corresponding NLU samples
advanced_nlu_df = generate_nlu_ground_truth(advanced_ir_df)
combined_nlu_df = pd.concat([nlu_df, advanced_nlu_df], ignore_index=True)
print(f"Total NLU samples: {len(combined_nlu_df)}")

# Save updated files
combined_ir_df[['query', 'relevant_tables']].to_csv(ir_output_path, index=False)
combined_nlu_df[['query', 'answer', 'entities']].to_csv(nlu_output_path, index=False)

print(f"\n✅ Updated files saved with enhanced samples")
print(f"IR samples: {len(combined_ir_df)}")
print(f"NLU samples: {len(combined_nlu_df)}")

## Summary and Next Steps

In [ ]:
print("=== Generation Summary ===")
print(f"📁 Generated files:")
print(f"   • sample_ir_ground_truth.csv - {len(combined_ir_df)} IR queries")
print(f"   • sample_nlu_ground_truth.csv - {len(combined_nlu_df)} NLU examples")
print(f"   • detailed_ir_analysis.csv - Full IR analysis")
print(f"   • detailed_nlu_analysis.csv - Full NLU analysis")

print(f"\n📊 Coverage:")
print(f"   • Categories: {len(combined_ir_df['category'].unique())} unique")
print(f"   • Tables: {len(set(','.join(combined_ir_df['relevant_tables']).split(',')))} referenced")
print(f"   • Avg entities/query: {combined_nlu_df['entities'].apply(lambda x: len(x.split(',')) if x else 0).mean():.1f}")

print(f"\n🎯 Quality metrics:")
print(f"   • Unique queries: {combined_ir_df['query'].nunique()}/{len(combined_ir_df)} ({100*combined_ir_df['query'].nunique()/len(combined_ir_df):.1f}%)")
print(f"   • Avg relevant tables: {combined_ir_df['num_relevant'].mean():.1f}")
print(f"   • Complex queries: {(combined_nlu_df['complexity'] == 'medium').sum()}/{len(combined_nlu_df)}")

print(f"\n🚀 Next steps:")
print(f"   1. Run the UI with 'Use generated sample data' option")
print(f"   2. Test academic benchmarking with the generated data")
print(f"   3. Fine-tune query patterns based on results")
print(f"   4. Generate domain-specific variants as needed")

print(f"\n✅ Data generation complete!")